In [ ]:
import xml.etree.ElementTree as ET
import xmltodict
import json
import os
import pandas as pd


data_cleaner_lookup = pd.read_csv('/home/sumit-tyagi/sumit-ssd/common-biscuit/main/planogram_biscuit/notebooks/eda/data_cleaning_requirement.csv')
data_cleaner_lookup = data_cleaner_lookup[['actual', 'required']].set_index('actual').to_dict()['required']




annotation_path = r'/home/sumit-tyagi/sumit-ssd/common-biscuit/main/planogram_biscuit/data/raw/annotations_master'
destination_path = os.path.join(annotation_path, 'cleaned')
os.mkdir(destination_path)

for i in os.walk(annotation_path):
    # print(i[2])
    files_effected = 0
    file_effected = False
    for j in i[2]:
        tree = ET.parse(os.path.join(annotation_path,j))
        xml_data = tree.getroot()
        for element in xml_data.findall('object'):
            
            if element.find('name').text in data_cleaner_lookup.keys():
                file_effected = True
                if data_cleaner_lookup[element.find('name').text] == 'Remove':
                    xml_data.remove(element)
                    
                else:
                    element.find('name').text = data_cleaner_lookup[element.find('name').text]


        if file_effected:
            files_effected += 1
            file_effected = False
    
        tree.write(os.path.join(destination_path, j))
print("Total files effected: ", files_effected)


In [14]:
from collections import defaultdict


brand_names_count = defaultdict(int)

for i in os.walk('/home/sumit-tyagi/sumit-ssd/common-biscuit/main/planogram_biscuit/data/raw/annotations_master/cleaned'):
    # print(i)
    for j in i[2]:
        tree = ET.parse('/home/sumit-tyagi/sumit-ssd/common-biscuit/main/planogram_biscuit/data/raw/annotations_master/cleaned/'+j)
        xml_data = tree.getroot()
        xmlstr = ET.tostring(xml_data, encoding='utf-8', method='xml')

        data_dict = dict(xmltodict.parse(xmlstr))
        try:
            for i in data_dict['annotation']['object']:
                brand_names_count[i['name']] += 1
                i['name'] = 'packet'
        except:
            print("No objects in:", j)


import pandas as pd
brand_names_list = list(brand_names_count.keys())
annotation_data = pd.DataFrame(data = list(brand_names_count.items()), columns= ['brand_name', 'annotation_count'])


No objects in: PHOTO-2021-05-23-08-50-27.xml
No objects in: PHOTO-2021-07-05-12-23-19.xml
No objects in: PHOTO-2021-07-08-14-38-07.xml
No objects in: PHOTO-2021-07-01-09-23-18.xml
No objects in: PHOTO-2021-07-08-12-47-52.xml
No objects in: PHOTO-2021-07-10-08-24-01.xml
No objects in: PHOTO-2021-07-03-13-13-08.xml
No objects in: PHOTO-2021-06-24-10-31-22.xml
No objects in: PHOTO-2021-05-14-12-56-20.xml
No objects in: PHOTO-2021-05-29-15-57-27.xml
No objects in: PHOTO-2021-07-15-10-14-15.xml
No objects in: PHOTO-2021-07-26-09-43-37.xml


In [15]:
df = pd.read_csv('/home/sumit-tyagi/sumit-ssd/common-biscuit/main/planogram_biscuit/data/lookups/products-updated.csv')
df.columns = ['brand_name', 'count']
df.head(2)

,brand_name,count
0,arcoiris_clasico_paquetin,1079
1,arcoiris_fruts_paquetin,1038


In [16]:
total_data = annotation_data.merge(df[['brand_name', 'count']], on='brand_name', how='outer' )

In [17]:
total_data[total_data['annotation_count'].isna()].sort_values(by='brand_name')

,brand_name,annotation_count,count
74,chokis_clasico_paquetin,NaN,1102.0


In [18]:
total_data[total_data['count'].isna()].sort_values(by='brand_name')

,brand_name,annotation_count,count
32,Other,867.0,NaN
44,barras_de_coco_paquetin_strip,8.0,NaN
33,bizcochitos_clasica_bolsa_strip,86.0,NaN
34,bombitos_clasica_bolsa_strip,80.0,NaN
59,chocolatines_paquetin_strip,70.0,NaN
46,chokis_brownie_paquetin_strip,19.0,NaN
62,chokis_chocotella_paquetin_strip,14.0,NaN
71,cremax_de_nieve_chocolate_strip,9.0,NaN
73,cremax_de_nieve_fresa_strip,8.0,NaN
72,cremax_de_nieve_vainilla_strip,5.0,NaN


In [16]:
#!/usr/bin/env python

import subprocess as sbp
import os

folder_lookup = [('barras_de_coco_paquetin_strip', 'barras_de_coco_paquetin'),
('chokis_chocotella_paquetin_strip', 'chokis_chocotella_paquetin'),
('cremax_de_nieve_chocolate_strip', 'cremax_de_nieve_chocolate'),
('cremax_de_nieve_fresa_strip', 'cremax_de_nieve_fresa'),
('cremax_de_nieve_vainilla_strip', 'cremax_de_nieve_vainilla'),
('delicias_bolsa_strip', 'delicias_bolsa'),
('flor_de_naranjo_paquetin_strip', 'flor_de_naranjo_paquetin'),
('maravillas_strip', 'maravillas'),
('marias_doradas_strip', 'marias_doradas'),
('mini_marias_chocolate_bolsa_strip', 'mini_marias_chocolate_bolsa'),
('mini_marias_bolsa_strip', 'mini_marias_bolsa')]


parent_folder_path = '/home/sumit-tyagi/sumit-ssd/common-biscuit/main/planogram_biscuit/data/cropped_images/1108_0threshold_0image_size_73folders/train'
for f2, f1 in folder_lookup:
    try:
        path_1= os.path.join(parent_folder_path, f1)
        previous_count = int(os.listdir(path_1)[-1].split()[1].split('.')[0])
        counter = previous_count + 1

        path_2 = os.path.join(parent_folder_path, f2)
        for image in os.listdir(path_2):
            sbp.call(['mv', os.path.join(path_2, image), os.path.join(path_1, 'Image '+ str(counter)+'.jpg')])
            counter += 1
        print('Moved', counter - previous_count, 'images from', f2, 'to', f1)
        os.rmdir(path_2)
    except Exception as e:
        print(e)




Moved 9 images from barras_de_coco_paquetin_strip to barras_de_coco_paquetin
Moved 14 images from chokis_chocotella_paquetin_strip to chokis_chocotella_paquetin
Moved 10 images from cremax_de_nieve_chocolate_strip to cremax_de_nieve_chocolate
Moved 9 images from cremax_de_nieve_fresa_strip to cremax_de_nieve_fresa
Moved 6 images from cremax_de_nieve_vainilla_strip to cremax_de_nieve_vainilla
Moved 5 images from delicias_bolsa_strip to delicias_bolsa
Moved 3 images from flor_de_naranjo_paquetin_strip to flor_de_naranjo_paquetin
Moved 210 images from maravillas_strip to maravillas
Moved 13 images from marias_doradas_strip to marias_doradas
Moved 93 images from mini_marias_chocolate_bolsa_strip to mini_marias_chocolate_bolsa
Moved 238 images from mini_marias_bolsa_strip to mini_marias_bolsa


In [ ]:
parent_folder_path = '/home/sumit-tyagi/sumit-ssd/common-biscuit/main/planogram_biscuit/data/cropped_images/1108_0threshold_0image_size_73folders/valid'
for f2, f1 in folder_lookup:
    try:
        path_1= os.path.join(parent_folder_path, f1)
        previous_count = int(os.listdir(path_1)[-1].split()[1].split('.')[0])
        counter = previous_count + 1

        path_2 = os.path.join(parent_folder_path, f2)
        for image in os.listdir(path_2):
            sbp.call(['mv', os.path.join(path_2, image), os.path.join(path_1, 'Image '+ str(counter)+'.jpg')])
            counter += 1
        print('Moved', counter - previous_count, 'images from', f2, 'to', f1)
        os.rmdir(path_2)
    except Exception as e:
        print(e)